<a href="https://colab.research.google.com/github/kavyajeetbora/foursquare_ai/blob/master/notebooks/07_duckdb_ai_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install --quiet duckdb jupysql duckdb-engine

In [19]:
import duckdb
import os
import pandas as pd

In [22]:
# name,
# address,
# latitude,
# longitude,
# region,
# postcode,
# fsq_category_labels as category,
# geom

In [55]:
# Initialize DuckDB connection
con = duckdb.connect()

# Load the httpfs extension to enable S3 access
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

s3_places_path = 's3://fsq-os-places-us-east-1/release/dt=2025-09-09/places/parquet/places-*.zstd.parquet'

s3_categories_path = 's3://fsq-os-places-us-east-1/release/dt=2025-09-09/categories/parquet/categories.zstd.parquet'

df = duckdb.sql(
    f"""
    WITH places AS (
        SELECT
            DISTINCT UNNEST(P.fsq_category_ids) as fsq_category_id,
            name,
            postcode,
            address,
            region,
            geom
        FROM read_parquet('{s3_places_path}') AS P
        WHERE latitude IS NOT NULL AND longitude IS NOT NULL AND country='IN'
    ),
    places_with_categories AS (
        SELECT
            P.name AS name,
            C.level1_category_name AS category_level_1,
            C.level2_category_name AS category_level_2,
            postcode,
            address,
            region,
            P.geom
        FROM places AS P
        JOIN read_parquet('{s3_categories_path}') AS C
        ON P.fsq_category_id = C.category_id
    )


    SELECT
        name,
        category_level_1,
        category_level_2,
        address,
        region,
        postcode,
        geom
    FROM places_with_categories LIMIT 10;

""")

print(df)

# Close the DuckDB connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────────────────────┬───────────────────────────┬───────────────────────────┬───────────────────────┬─────────────────────────────┬──────────┬───────────────────────────────────────────────────────────────────────┐
│           name            │     category_level_1      │     category_level_2      │        address        │           region            │ postcode │                                 geom                                  │
│          varchar          │          varchar          │          varchar          │        varchar        │           varchar           │ varchar  │                                 blob                                  │
├───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────┼─────────────────────────────┼──────────┼───────────────────────────────────────────────────────────────────────┤
│ Little Andaman Lighthouse │ Landmarks and Outdoors    │ Lighthouse                │ Little Andaman Island 

In [45]:
# Initialize DuckDB connection
con = duckdb.connect()

# Load the httpfs extension to enable S3 access
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

s3_path = 's3://fsq-os-places-us-east-1/release/dt=2025-09-09/categories/parquet/categories.zstd.parquet'
df = duckdb.sql(
    f"""
        SELECT *
        FROM read_parquet('{s3_path}') LIMIT 10;
""")

print(df)

# Close the DuckDB connection
con.close()

┌──────────────────────────┬────────────────┬───────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────┬──────────────────────────┬──────────────────────────┬─────────────────────────────┬──────────────────────────┬───────────────────────────┬──────────────────────────┬───────────────────────────┬──────────────────────────┬──────────────────────┬──────────────────────────┬──────────────────────┬────────────┐
│       category_id        │ category_level │     category_name     │                                                            category_label                                                            │    level1_category_id    │   level1_category_name   │    level2_category_id    │    level2_category_name     │    level3_category_id    │   level3_category_name    │    level4_category_id    │   level4_category_name    │    level5_category_id    │ level5_c